Do the market basket analysis with Bread_Basket dataset.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os

# for market basket analysis
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
os.chdir('/Users/gowthamshashank/Desktop/LU/LetsUpgrade/ass-22')

df = pd.read_csv('BreadBasket_DMS - Arindam Dev.csv',encoding = 'unicode_escape')

df.head()


,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
2,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam


In [3]:
df.shape

(21293, 4)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21293 entries, 0 to 21292
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         21293 non-null  object
 1   Time         21293 non-null  object
 2   Transaction  21293 non-null  int64 
 3   Item         21293 non-null  object
dtypes: int64(1), object(3)
memory usage: 665.5+ KB


In [5]:
df.isnull().sum()

Date           0
Time           0
Transaction    0
Item           0
dtype: int64

In [6]:
df.nunique()

Date            159
Time           8240
Transaction    9531
Item             95
dtype: int64

In [7]:
df.columns

Index(['Date', 'Time', 'Transaction', 'Item'], dtype='object')

In [8]:
df[df['Transaction']<0].head()

,Date,Time,Transaction,Item


In [9]:
df['Date'] = df['Date'].astype('str')

In [10]:
df = df[~df['Date'].str.contains('C',na=False)]

In [11]:
df['Date']

0        2016-10-30
1        2016-10-30
2        2016-10-30
3        2016-10-30
4        2016-10-30
            ...    
21288    2017-04-09
21289    2017-04-09
21290    2017-04-09
21291    2017-04-09
21292    2017-04-09
Name: Date, Length: 21293, dtype: object

In [12]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Hour'] = df['Date'].dt.hour

In [13]:
df.head()

,Date,Time,Transaction,Item,Year,Month,Hour
0,2016-10-30,09:58:11,1,Bread,2016,10,0
1,2016-10-30,10:05:34,2,Scandinavian,2016,10,0
2,2016-10-30,10:05:34,2,Scandinavian,2016,10,0
3,2016-10-30,10:07:57,3,Hot chocolate,2016,10,0
4,2016-10-30,10:07:57,3,Jam,2016,10,0


In [14]:
df['Month_year'] = pd.to_datetime(df['Date']).dt.to_period('M')
df.head()

,Date,Time,Transaction,Item,Year,Month,Hour,Month_year
0,2016-10-30,09:58:11,1,Bread,2016,10,0,2016-10
1,2016-10-30,10:05:34,2,Scandinavian,2016,10,0,2016-10
2,2016-10-30,10:05:34,2,Scandinavian,2016,10,0,2016-10
3,2016-10-30,10:07:57,3,Hot chocolate,2016,10,0,2016-10
4,2016-10-30,10:07:57,3,Jam,2016,10,0,2016-10


In [15]:
df.describe()

,Transaction,Year,Month,Hour
count,21293.000000,21293.000000,21293.000000,21293.0
mean,4951.990889,2016.597943,5.904898,0.0
std,2787.758400,0.490325,4.557956,0.0
min,1.000000,2016.000000,1.000000,0.0
25%,2548.000000,2016.000000,2.000000,0.0
50%,5067.000000,2017.000000,3.000000,0.0
75%,7329.000000,2017.000000,11.000000,0.0
max,9684.000000,2017.000000,12.000000,0.0


In [16]:
df['Item'].unique()

array(['Bread', 'Scandinavian', 'Hot chocolate', 'Jam', 'Cookies',
       'Muffin', 'Coffee', 'Pastry', 'Medialuna', 'Tea', 'NONE',
       'Tartine', 'Basket', 'Mineral water', 'Farm House', 'Fudge',
       'Juice', "Ella's Kitchen Pouches", 'Victorian Sponge', 'Frittata',
       'Hearty & Seasonal', 'Soup', 'Pick and Mix Bowls', 'Smoothies',
       'Cake', 'Mighty Protein', 'Chicken sand', 'Coke',
       'My-5 Fruit Shoot', 'Focaccia', 'Sandwich', 'Alfajores', 'Eggs',
       'Brownie', 'Dulce de Leche', 'Honey', 'The BART', 'Granola',
       'Fairy Doors', 'Empanadas', 'Keeping It Local', 'Art Tray',
       'Bowl Nic Pitt', 'Bread Pudding', 'Adjustment', 'Truffles',
       'Chimichurri Oil', 'Bacon', 'Spread', 'Kids biscuit', 'Siblings',
       'Caramel bites', 'Jammie Dodgers', 'Tiffin', 'Olum & polenta',
       'Polenta', 'The Nomad', 'Hack the stack', 'Bakewell',
       'Lemon and coconut', 'Toast', 'Scone', 'Crepes', 'Vegan mincepie',
       'Bare Popcorn', 'Muesli', 'Crisps', 'Pi

In [17]:
df['Item'].value_counts().head(10)

Coffee           5471
Bread            3325
Tea              1435
Cake             1025
Pastry            856
NONE              786
Sandwich          771
Medialuna         616
Hot chocolate     590
Cookies           540
Name: Item, dtype: int64

In [18]:
df[df['Item']=='Coffee'].head()

,Date,Time,Transaction,Item,Year,Month,Hour,Month_year
7,2016-10-30,10:13:03,5,Coffee,2016,10,0,2016-10
15,2016-10-30,10:19:12,7,Coffee,2016,10,0,2016-10
28,2016-10-30,10:30:14,12,Coffee,2016,10,0,2016-10
34,2016-10-30,10:31:24,13,Coffee,2016,10,0,2016-10
44,2016-10-30,10:37:08,16,Coffee,2016,10,0,2016-10


In [19]:
desc = df['Item'].value_counts().head(20)
desc

Coffee           5471
Bread            3325
Tea              1435
Cake             1025
Pastry            856
NONE              786
Sandwich          771
Medialuna         616
Hot chocolate     590
Cookies           540
Brownie           379
Farm House        374
Muffin            370
Juice             369
Alfajores         369
Soup              342
Scone             327
Toast             318
Scandinavian      277
Truffles          193
Name: Item, dtype: int64

In [20]:
df['Transaction'].nunique()

9531

In [21]:
df[df['Transaction'] == '00000'].head()

/Users/gowthamshashank/opt/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,Date,Time,Transaction,Item,Year,Month,Hour,Month_year


In [22]:
data = df[df['Item'] == 'Coffee']
data.head()

,Date,Time,Transaction,Item,Year,Month,Hour,Month_year
7,2016-10-30,10:13:03,5,Coffee,2016,10,0,2016-10
15,2016-10-30,10:19:12,7,Coffee,2016,10,0,2016-10
28,2016-10-30,10:30:14,12,Coffee,2016,10,0,2016-10
34,2016-10-30,10:31:24,13,Coffee,2016,10,0,2016-10
44,2016-10-30,10:37:08,16,Coffee,2016,10,0,2016-10


In [23]:
basket = data.groupby(['Date','Item'])['Transaction'].sum().unstack().fillna(0)
basket

Item,Coffee
Date,
2016-10-30,1541
2016-10-31,7496
2016-11-01,8167
2016-11-02,12156
2016-11-03,15502
...,...
2017-04-05,282472
2017-04-06,255865
2017-04-07,276214


In [24]:
basket_model = basket.applymap(lambda x: 1 if x>0 else 0)
basket_model

Item,Coffee
Date,
2016-10-30,1
2016-10-31,1
2016-11-01,1
2016-11-02,1
2016-11-03,1
...,...
2017-04-05,1
2017-04-06,1
2017-04-07,1


In [25]:
popular_sets = apriori(basket_model, min_support=0.08, use_colnames=True)

In [26]:
popular_sets

,support,itemsets
0,1.0,(Coffee)


In [27]:
rules = association_rules(popular_sets,metric='lift',min_threshold=1)

In [28]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [29]:
rules[ (rules['lift'] >= 5) &
       (rules['confidence'] >= 1) ]


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
